<a href="https://colab.research.google.com/github/121deepti/Cardiovascular_Risk_Prediction/blob/main/Zomato_Restaurant_Clustering_and_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project Name**    - Zomato Restaurant Clustering and Sentiment Analysis



##### **Project Type**    - Unsupervised
##### **Contribution**    - Individual

# **Project Summary -**

Write the summary here within 500-600 words.

# **GitHub Link -**

Provide your GitHub Link here.

# **Problem Statement**


**Write Problem Statement Here.**

# **General Guidelines** : -  

1.   Well-structured, formatted, and commented code is required.
2.   Exception Handling, Production Grade Code & Deployment Ready Code will be a plus. Those students will be awarded some additional credits.
     
     The additional credits will have advantages over other students during Star Student selection.
       
             [ Note: - Deployment Ready Code is defined as, the whole .ipynb notebook should be executable in one go
                       without a single error logged. ]

3.   Each and every logic should have proper comments.
4. You may add as many number of charts you want. Make Sure for each and every chart the following format should be answered.
        

```
# Chart visualization code
```
            

*   Why did you pick the specific chart?
*   What is/are the insight(s) found from the chart?
* Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

5. You have to create at least 15 logical & meaningful charts having important insights.


[ Hints : - Do the Vizualization in  a structured way while following "UBM" Rule.

U - Univariate Analysis,

B - Bivariate Analysis (Numerical - Categorical, Numerical - Numerical, Categorical - Categorical)

M - Multivariate Analysis
 ]





6. You may add more ml algorithms for model creation. Make sure for each and every algorithm, the following format should be answered.


*   Explain the ML Model used and it's performance using Evaluation metric Score Chart.


*   Cross- Validation & Hyperparameter Tuning

*   Have you seen any improvement? Note down the improvement with updates Evaluation metric Score Chart.

*   Explain each evaluation metric's indication towards business and the business impact pf the ML model used.




















# ***Let's Begin !***

## ***1. Know Your Data***

### Import Libraries

In [ ]:
# Import Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import missingno as msno

### Dataset Loading

In [ ]:
# Load Dataset
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
hotel_df=pd.read_csv("/content/drive/MyDrive/my_data/Zomato Restaurant names and Metadata.csv")
review_df=pd.read_csv("/content/drive/MyDrive/my_data/Zomato Restaurant reviews.csv")

### Dataset First View

In [ ]:
# Restaurant Dataset First Look
hotel_df.sample(5)

In [ ]:
# Review Dataset First Look
review_df.sample(5)

### Dataset Rows & Columns count

In [ ]:
# Restaurant Dataset Rows & Columns count
hotel_df.shape

In [ ]:
# Reviews Dataset Rows & Columns count
review_df.shape

### Dataset Information

In [ ]:
# Restaurant Dataset Info
hotel_df.info()

In [ ]:
# Reviews Dataset Reviews info
review_df.info()

#### Duplicate Values

In [ ]:
# Hotel Dataset Duplicate Value Count
hotel_df.duplicated().sum()

In [ ]:
# Reviews Dataset Duplicate Value Count
review_df.duplicated().sum()

In [ ]:
#Duplicates records of Reviews Dataset
review_df[review_df.duplicated()]

In [ ]:
#Removing Duplicates from Review Dataset
review_df=review_df.drop_duplicates()
review_df.duplicated().sum()

#### Missing Values/Null Values

In [ ]:
# Missing Values/Null Values Count of Restaurant dataset
hotel_df.isna().sum()

In [ ]:
# Missing Values/Null Values Count of User Review dataset
review_df.isna().sum()

In [ ]:
# Visualizing the missing values of Restaurant dataset
msno.bar(df=hotel_df,sort='ascending',figsize=(3,3),fontsize=8)

In [ ]:
# Visualizing the missing values of Review dataset
msno.bar(df=review_df,sort='ascending',figsize=(3,3),fontsize=8)

### What did you know about your dataset?

**Restaurant DataSet**

There are 105 total observation with 6 different features.
Feature like timing has 1 but collection has more than 50% null values.
There is no duplicate values i.e., 105 unique data.
Feature cost represent amount but has object data type because these values are separated by comma ','.
Timing represent operational hour but as it is represented in the form of text has object data type.

**Review DataSet**

There are total 10000 observation and 7 features.
Except picture and restaurant feature all others have null values.
There are total of 36 duplicate values for two restaurant - American Wild Wings and Arena Eleven, where all these duplicate values generally have null values that's why i have removed those duplicate records.
Rating represent ordinal data, has object data type should be integer.
Timing represent the time when review was posted but show object data type, it should be converted into date time.


## ***2. Understanding Your Variables***

In [ ]:
# Restaurant Dataset Columns
hotel_df.columns

In [ ]:
# Review Dataset Columns
review_df.columns

In [ ]:
# Restaurant Dataset Describe
hotel_df.describe(include='all')

In [ ]:
# Review Dataset Describe
review_df.describe(include='all')

### Variables Description

**Zomato Restaurant**

Name : Name of Restaurant

Links : URL Links of Restaurants

Cost : Per person estimated Cost of dining

Collection : Tagging of Restaurants w.r.t. Zomato categories

Cuisines : Cuisines served by Restaurants

Timings : Restaurant Timings


**Zomato Restaurant Reviews**


Restaurant : Name of the Restaurant

Reviewer : Name of the Reviewer

Review : Review Text

Rating : Rating Provided by Reviewer

MetaData : Reviewer Metadata - No. of Reviews and followers

Time: Date and Time of Review

Pictures : No. of pictures posted with review

### Check Unique Values for each variable.

In [ ]:
# Check Unique Values for each variable of Zomato dataset.
for i in hotel_df.columns:
  col=hotel_df[i]
  print("Number of Unique values in",i,"is:",col.nunique())

In [ ]:
# Check Unique Values for each variable of Zomato Reviews dataset.
for i in review_df.columns:
  col=review_df[i]
  print("Number of Unique values in",i,"is:",col.nunique())

## 3. ***Data Wrangling***

### Data Wrangling Code

In [ ]:
# Write your code to make your dataset analysis ready.
#Creating copies of datasets for performing eda
hotel_cpy=hotel_df.copy()
review_cpy=review_df.copy()

In [ ]:
# Remove commas from the 'Cost' column and convert it to numeric
hotel_cpy['Cost'] = hotel_cpy['Cost'].str.replace(',', '').astype(int)
#Creating bins of cost column and categorize cost column
hotel_cpy['Cost_bins']=pd.cut(hotel_cpy['Cost'],bins=[100,500,1000,2000,3000])
print('[Cost per person][Number of Restaurants]')
hotel_cpy['Cost_bins'].value_counts()

In [ ]:
#top 5 costlier restaurant
hotel_cpy.sort_values('Cost', ascending = False)[['Name','Cost']][:5]

In [ ]:
#top 5 economical restaurant
hotel_cpy.sort_values('Cost')[['Name','Cost']][:5]

In [ ]:
# spliting the Collections after removing Nulls and storing unique collection in dictionary
Collections_Values=hotel_cpy['Collections'].dropna().str.split(', ')
Collections_Values_set=set(Collections_Values.explode())
collection_dict=dict.fromkeys(Collections_Values_set,0)

In [ ]:
# Updating Collection dictionary with each collection value
def collection(tag):
  tag_list=tag.split(', ')
  for t in tag_list:
    if t in collection_dict:
      collection_dict[t]+=1

In [ ]:
#Removing nulls from collections column
Collections=hotel_cpy['Collections'].dropna()

#Calling collection method
Collections.apply(collection)

#sort the collection dict based on value in descending order
sorted_dict_collections = dict(sorted(collection_dict.items(), reverse=True,key=lambda item: item[1]))

# Slice the first 5 collection
top_5_collections = list(sorted_dict_collections.items())[:5]

# Print the top 5 collections with their frequency
print("Collections-Number of Restaurants")
for key, value in top_5_collections:
    print(key,'-', value)

In [ ]:
# spliting the Cuisines column and storing unique Cuisines in dictionary
Cuisines_Values=hotel_cpy['Cuisines'].str.split(', ')
Cuisines_Values_set=set(Cuisines_Values.explode())
Cuisines_dict=dict.fromkeys(Cuisines_Values_set,0)

In [ ]:
# Updating Cuisines dictionary with each Cuisines value
def Cuisines(val):
  val_list=val.split(', ')
  for t in val_list:
    if t in Cuisines_dict:
      Cuisines_dict[t]+=1

In [ ]:
#Calling Cuisines method
hotel_cpy.Cuisines.apply(Cuisines)

#sort the Cuisines dict based on value in descending order
sorted_dict_Cuisines = dict(sorted(Cuisines_dict.items(), reverse=True,key=lambda item: item[1]))

# Slice the first 5 Cuisines
top_5_Cuisines = list(sorted_dict_Cuisines.items())[:5]

# Print the top 5 Cuisiness with their frequency
print("Cusines-Number of Restaurants")
for key, value in top_5_Cuisines:
    print(key,'-',value)

In [ ]:
#Categorize Collections based on average cost
hotel_cpy.groupby(["Collections"])["Cost"].mean().sort_values(ascending=False).reset_index()

In [ ]:
#Categorize Cuisines based on average cost
hotel_cpy.groupby(["Cuisines"])["Cost"].mean().sort_values(ascending=False).reset_index()

In [ ]:
#Shows how many times a restaurnt got an review
review_cpy.groupby('Restaurant')['Reviewer'].count().sort_values(ascending=False).reset_index()

In [ ]:
#Removing Null values from ratings column
review_cpy.Rating=review_cpy.Rating.dropna()

#Replacing char value of'Like' with median value for futher analysis
median_df=review_cpy[review_cpy.Rating!='Like']
median=median_df.Rating.median()
review_cpy.Rating=review_cpy.Rating.replace('Like',median)

#Changing data type to float
review_cpy.Rating=review_cpy.Rating.astype(float)

In [ ]:
#Dropping Null values from metadata column
review_cpy.dropna(subset=['Metadata'],inplace=True)

In [ ]:
#Splitting reviewer's metadata column into total review and total follwers of that reviewer
review_cpy['Reviewer_Total_Review'],review_cpy['Reviewer_Followers']=review_cpy['Metadata'].str.split(',').str

#fetching number from the text and convert it to numeric data type
review_cpy['Reviewer_Total_Review'] = pd.to_numeric(review_cpy['Reviewer_Total_Review'].str.split(' ').str[0])
review_cpy['Reviewer_Followers'] = pd.to_numeric(review_cpy['Reviewer_Followers'].str.split(' ').str[1])

#Converting the time column to datetime type and fetch year, month from it
review_cpy['Time']=pd.to_datetime(review_cpy['Time'])
review_cpy['Review_Year'] = pd.DatetimeIndex(review_cpy['Time']).year
review_cpy['Review_Month'] = pd.DatetimeIndex(review_cpy['Time']).month
#review_cpy['Review_Hour'] = pd.DatetimeIndex(review_cpy['Time']).hour

In [ ]:
review_cpy.head()

In [ ]:
#The top 10 Reviewers and their rating which have a large number of followers
review_cpy.groupby('Reviewer').agg({'Reviewer_Total_Review':max,'Reviewer_Followers':max,'Rating':'mean'}).round({'Rating': 1}).sort_values(by='Reviewer_Followers',ascending=False).head(10)

In [ ]:
#finding hotel without review
non_reviewed_hotel_list=[i for i in review_cpy.Restaurant]
hotel_list=[i for i in hotel_cpy.Name if i not in non_reviewed_hotel_list]
print(hotel_list)

In [ ]:
#Finding top 5 reviewers based on reviews posted
review_cpy.groupby('Reviewer')['Review'].count().sort_values(ascending=False).reset_index().head(5)

In [ ]:
#fetching which year and month are in reviewer's focus
review_cpy.groupby(['Review_Year','Review_Month'])['Review'].count().sort_values(ascending=False).reset_index().rename(columns={'Review':'Total Reviews'})

In [ ]:
#Merging the hotel and review datasets
merged_df = pd.merge(hotel_cpy, review_cpy, left_on='Name', right_on='Restaurant', how='inner')
merged_df.shape

In [ ]:
#top 5 highly rated restaurant with their cost
merged_df.groupby('Name').agg({'Rating':'mean','Cost':'mean'}).round(1).sort_values(by='Rating',ascending=False).reset_index().head(5)

In [ ]:
#top 5 low rated restaurant with their cost
merged_df.groupby('Name').agg({'Rating':'mean','Cost':'mean'}).round(1).sort_values(by='Rating').reset_index().head(5)

### What all manipulations have you done and insights you found?

Firstly, I started with changing data types for cost and rating. In rating there was only one rating which was string or has value of like so I change it into median of the rating. This was done to make data consistent.

**Restaurant dataset:**first I binned cost column and found that there are around 40 restaurant in the cost range of 500-1000 and only 2 restaurant in the cost range of 2000-3000.
I also figured out 5 costlier restaurant in which Collage - Hyatt Hyderabad Gachibowli has maximum price of 2800 and then found the lowest which is Amul and Mohammedia Shawarma with price of 150.North indian cuisine is preferred choice with Great Buffet tags is mostly used in hotels.
The restaurant tag with Sunday Brunch is on the top based on price.
Continental, Italian, North Indian, Chinese, Asian cuisines restaurants have the highest cost 2800.

**Review dataset:** There are around 98 restaurants which are reviewed by 100 reviewers.After splitting reviewer's metadata column into total review and total follwers of that reviewer I found that the most followed critic which was Satwinder Singh who posted total of 186 reviews and had followers of 13410 who gives an average of 3.67 rating for each order he makes.
I have also converted the time column to datetime type and fetch year and month from it.There are some restaurants which are not reviewed at all.
Parijat Ray and Ankita posted the highest number of reviews.On May,2019 the highest number of total 1347 reviews are posted.

Then I merged the two dataset together to figure out the price point for the restaurant, top rated restaurant AB's - Absolute Barbecues has a price point of 1500 and the low rated Hotel Zara Hi-Fi has price point of 400.

In order to exactly understand why even with price point of 1500 these hotel has maximum number of rating and sentiment of those rating need to extract words from the text and do futher analysis of the review and then followed by forming clusters so that one can get recommendation about top quality restaurants.

## ***4. Data Vizualization, Storytelling & Experimenting with charts : Understand the relationships between variables***

#### Chart - 1

In [ ]:
# Chart - 1 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 2

In [ ]:
# Chart - 2 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 3

In [ ]:
# Chart - 3 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 4

In [ ]:
# Chart - 4 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 5

In [ ]:
# Chart - 5 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 6

In [ ]:
# Chart - 6 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 7

In [ ]:
# Chart - 7 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 8

In [ ]:
# Chart - 8 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 9

In [ ]:
# Chart - 9 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 10

In [ ]:
# Chart - 10 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 11

In [ ]:
# Chart - 11 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 12

In [ ]:
# Chart - 12 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 13

In [ ]:
# Chart - 13 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 14 - Correlation Heatmap

In [ ]:
# Correlation Heatmap visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

#### Chart - 15 - Pair Plot

In [ ]:
# Pair Plot visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

## ***5. Hypothesis Testing***

### Based on your chart experiments, define three hypothetical statements from the dataset. In the next three questions, perform hypothesis testing to obtain final conclusion about the statements through your code and statistical testing.

Answer Here.

### Hypothetical Statement - 1

#### 1. State Your research hypothesis as a null hypothesis and alternate hypothesis.

Answer Here.

#### 2. Perform an appropriate statistical test.

In [ ]:
# Perform Statistical Test to obtain P-Value

##### Which statistical test have you done to obtain P-Value?

Answer Here.

##### Why did you choose the specific statistical test?

Answer Here.

### Hypothetical Statement - 2

#### 1. State Your research hypothesis as a null hypothesis and alternate hypothesis.

Answer Here.

#### 2. Perform an appropriate statistical test.

In [ ]:
# Perform Statistical Test to obtain P-Value

##### Which statistical test have you done to obtain P-Value?

Answer Here.

##### Why did you choose the specific statistical test?

Answer Here.

### Hypothetical Statement - 3

#### 1. State Your research hypothesis as a null hypothesis and alternate hypothesis.

Answer Here.

#### 2. Perform an appropriate statistical test.

In [ ]:
# Perform Statistical Test to obtain P-Value

##### Which statistical test have you done to obtain P-Value?

Answer Here.

##### Why did you choose the specific statistical test?

Answer Here.

## ***6. Feature Engineering & Data Pre-processing***

### 1. Handling Missing Values

In [ ]:
# Handling Missing Values & Missing Value Imputation

#### What all missing value imputation techniques have you used and why did you use those techniques?

Answer Here.

### 2. Handling Outliers

In [ ]:
# Handling Outliers & Outlier treatments

##### What all outlier treatment techniques have you used and why did you use those techniques?

Answer Here.

### 3. Categorical Encoding

In [ ]:
# Encode your categorical columns

#### What all categorical encoding techniques have you used & why did you use those techniques?

Answer Here.

### 4. Textual Data Preprocessing
(It's mandatory for textual dataset i.e., NLP, Sentiment Analysis, Text Clustering etc.)

#### 1. Expand Contraction

In [ ]:
# Expand Contraction

#### 2. Lower Casing

In [ ]:
# Lower Casing

#### 3. Removing Punctuations

In [ ]:
# Remove Punctuations

#### 4. Removing URLs & Removing words and digits contain digits.

In [ ]:
# Remove URLs & Remove words and digits contain digits

#### 5. Removing Stopwords & Removing White spaces

In [ ]:
# Remove Stopwords

In [ ]:
# Remove White spaces

#### 6. Rephrase Text

In [ ]:
# Rephrase Text

#### 7. Tokenization

In [ ]:
# Tokenization

#### 8. Text Normalization

In [ ]:
# Normalizing Text (i.e., Stemming, Lemmatization etc.)

##### Which text normalization technique have you used and why?

Answer Here.

#### 9. Part of speech tagging

In [ ]:
# POS Taging

#### 10. Text Vectorization

In [ ]:
# Vectorizing Text

##### Which text vectorization technique have you used and why?

Answer Here.

### 4. Feature Manipulation & Selection

#### 1. Feature Manipulation

In [ ]:
# Manipulate Features to minimize feature correlation and create new features

#### 2. Feature Selection

In [ ]:
# Select your features wisely to avoid overfitting

##### What all feature selection methods have you used  and why?

Answer Here.

##### Which all features you found important and why?

Answer Here.

### 5. Data Transformation

#### Do you think that your data needs to be transformed? If yes, which transformation have you used. Explain Why?

In [ ]:
# Transform Your data

### 6. Data Scaling

In [ ]:
# Scaling your data

##### Which method have you used to scale you data and why?

### 7. Dimesionality Reduction

##### Do you think that dimensionality reduction is needed? Explain Why?

Answer Here.

In [ ]:
# DImensionality Reduction (If needed)

##### Which dimensionality reduction technique have you used and why? (If dimensionality reduction done on dataset.)

Answer Here.

### 8. Data Splitting

In [ ]:
# Split your data to train and test. Choose Splitting ratio wisely.

##### What data splitting ratio have you used and why?

Answer Here.

### 9. Handling Imbalanced Dataset

##### Do you think the dataset is imbalanced? Explain Why.

Answer Here.

In [ ]:
# Handling Imbalanced Dataset (If needed)

##### What technique did you use to handle the imbalance dataset and why? (If needed to be balanced)

Answer Here.

## ***7. ML Model Implementation***

### ML Model - 1

In [ ]:
# ML Model - 1 Implementation

# Fit the Algorithm

# Predict on the model

#### 1. Explain the ML Model used and it's performance using Evaluation metric Score Chart.

In [ ]:
# Visualizing evaluation Metric Score chart

#### 2. Cross- Validation & Hyperparameter Tuning

In [ ]:
# ML Model - 1 Implementation with hyperparameter optimization techniques (i.e., GridSearch CV, RandomSearch CV, Bayesian Optimization etc.)

# Fit the Algorithm

# Predict on the model

##### Which hyperparameter optimization technique have you used and why?

Answer Here.

##### Have you seen any improvement? Note down the improvement with updates Evaluation metric Score Chart.

Answer Here.

### ML Model - 2

#### 1. Explain the ML Model used and it's performance using Evaluation metric Score Chart.

In [ ]:
# Visualizing evaluation Metric Score chart

#### 2. Cross- Validation & Hyperparameter Tuning

In [ ]:
# ML Model - 1 Implementation with hyperparameter optimization techniques (i.e., GridSearch CV, RandomSearch CV, Bayesian Optimization etc.)

# Fit the Algorithm

# Predict on the model

##### Which hyperparameter optimization technique have you used and why?

Answer Here.

##### Have you seen any improvement? Note down the improvement with updates Evaluation metric Score Chart.

Answer Here.

#### 3. Explain each evaluation metric's indication towards business and the business impact pf the ML model used.

Answer Here.

### ML Model - 3

In [ ]:
# ML Model - 3 Implementation

# Fit the Algorithm

# Predict on the model

#### 1. Explain the ML Model used and it's performance using Evaluation metric Score Chart.

In [ ]:
# Visualizing evaluation Metric Score chart

#### 2. Cross- Validation & Hyperparameter Tuning

In [ ]:
# ML Model - 3 Implementation with hyperparameter optimization techniques (i.e., GridSearch CV, RandomSearch CV, Bayesian Optimization etc.)

# Fit the Algorithm

# Predict on the model

##### Which hyperparameter optimization technique have you used and why?

Answer Here.

##### Have you seen any improvement? Note down the improvement with updates Evaluation metric Score Chart.

Answer Here.

### 1. Which Evaluation metrics did you consider for a positive business impact and why?

Answer Here.

### 2. Which ML model did you choose from the above created models as your final prediction model and why?

Answer Here.

### 3. Explain the model which you have used and the feature importance using any model explainability tool?

Answer Here.

## ***8.*** ***Future Work (Optional)***

### 1. Save the best performing ml model in a pickle file or joblib file format for deployment process.


In [ ]:
# Save the File

### 2. Again Load the saved model file and try to predict unseen data for a sanity check.


In [ ]:
# Load the File and predict unseen data.

### ***Congrats! Your model is successfully created and ready for deployment on a live server for a real user interaction !!!***

# **Conclusion**

Write the conclusion here.

### ***Hurrah! You have successfully completed your Machine Learning Capstone Project !!!***